# 1. Выбор датасета

In [1]:
from sklearn.datasets import load_wine

X, y = load_wine(return_X_y=True, as_frame=True)
classes = set(y)

classes

{0, 1, 2}

In [2]:
import pandas as pd 

data = load_wine(as_frame=True)

df = pd.DataFrame(data=data['data'], columns = data['feature_names'])
df.to_csv('dataset.csv', sep = ',', index = False)

# 2. Формирование выборок данных

In [3]:
df = X
df['y_true'] = y
bts = []
for i in range(3):
  df_bt = df.sample(frac=0.5)
  y_bt = df_bt['y_true']
  X_bt = df_bt.drop(columns=['y_true'])
  bts.append([X_bt, y_bt])

# 3. Выбор классификаторов

In [4]:
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import LinearSVC

models = [GaussianNB(), QuadraticDiscriminantAnalysis(), LinearSVC()]

# 4. Обучение классификаторов

In [5]:
import pandas as pd
from sklearn.metrics import accuracy_score

preds = []
for i, model in enumerate(models):
  X_bt = bts[i][0]
  y_bt = bts[i][1]
  model.fit(X_bt, y_bt)
  y_pred = model.predict(X_bt)
  pred_df = pd.DataFrame(y_pred, index=X_bt.index)
  preds.append(pred_df)
  print(accuracy_score(y_bt, y_pred))



0.9775280898876404
1.0
0.8089887640449438


/usr/local/lib/python3.8/dist-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


# 5. Объединение датасетов с предсказаниями

In [6]:
preds_df = pd.concat(preds, axis=1)
preds_df.head()

,0,0,0
0,0.0,0.0,1.0
1,0.0,NaN,0.0
2,NaN,0.0,NaN
3,0.0,NaN,0.0
4,0.0,NaN,1.0


In [7]:
preds_df.to_csv('labels.csv', sep = ',', index = True)

# 6. Формирование онтологии

In [8]:
!pip install owlready2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.5/25.5 MB 48.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for owlready2: filename=Owlready2-0.39-cp38-cp38-linux_x86_64.whl size=22260917 sha256=1ce192a7b47c23861979164822927514ea0325c0da8617af7e21d2aa262030d7
  Stored in directory: /root/.cache/pip/wheels/bd/16/5f/a0bfc34a8f7682cbd6b4d9cb1436c0a0a04ac3579394d7e28a
Successfully built owlready2


In [9]:
from owlready2 import *
onto = get_ontology('onto.owl')

In [10]:
import types

with onto:
  tsr = types.new_class('tsr', (ObjectProperty, TransitiveProperty, SymmetricProperty, ReflexiveProperty))
  ts = types.new_class('ts', (ObjectProperty, TransitiveProperty, SymmetricProperty))

  for cl in classes:
    onto_cl = types.new_class(f'Class{cl}', (Thing,))
    onto_cl_know = types.new_class(f'Know_Class{cl}', (Thing,))
    onto_cl_know.equivalent_to.append(tsr.only(onto_cl))
    onto_cl_believe = types.new_class(f'Believe_Class{cl}', (Thing,))
    onto_cl_believe.equivalent_to.append(ts.only(onto_cl))

onto.save('onto.owl', format='ntriples')



# 7. Создание индивидов для предсказаний
# 8. Установление неразличимости между предсказаниями для одного и того же объекта

In [11]:
with onto:
  for index, row in preds_df.iterrows():
    inst_old = None
    for j, pred in enumerate(preds_df.columns):
      cl = row.iloc[j]
      if not pd.isna(cl):
        cl = int(cl)
        onto_cl = onto[f'Class{cl}']
        inst_new = onto_cl(f'inst{index}_{j}')
        if inst_old != None:
          inst_old.tsr.append(inst_new)
          inst_old.ts.append(inst_new)
        inst_old = inst_new

onto.save('onto.owl', format='ntriples')


# 9. Ризонер для вывода отношений

In [12]:
with onto:
  sync_reasoner_pellet(infer_property_values=True, debug=2)

onto.save('reasoned.owl', format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-arq-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xml-apis-1.4.01.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-api-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/commons-codec-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpcore-4.2.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/usr/local/lib/p

* Owlready * Adding relation onto.inst95_1 tsr onto.inst95_1
* Owlready * Adding relation onto.inst30_1 tsr onto.inst30_1
* Owlready * Adding relation onto.inst30_1 ts onto.inst30_1
* Owlready * Adding relation onto.inst73_1 tsr onto.inst73_1
* Owlready * Adding relation onto.inst23_2 tsr onto.inst23_2
* Owlready * Adding relation onto.inst129_1 tsr onto.inst129_1
* Owlready * Adding relation onto.inst129_1 ts onto.inst129_1
* Owlready * Adding relation onto.inst105_0 tsr onto.inst105_0
* Owlready * Adding relation onto.inst105_0 ts onto.inst105_0
* Owlready * Adding relation onto.inst173_0 tsr onto.inst173_0
* Owlready * Adding relation onto.inst173_0 ts onto.inst173_0
* Owlready * Adding relation onto.inst6_2 tsr onto.inst6_2
* Owlready * Adding relation onto.inst100_1 tsr onto.inst100_1
* Owlready * Adding relation onto.inst100_1 ts onto.inst100_1
* Owlready * Adding relation onto.inst176_1 tsr onto.inst176_1
* Owlready * Adding relation onto.inst176_1 ts onto.inst176_1
* Owlready *

* Owlready2 * Pellet took 3.0719385147094727 seconds
* Owlready2 * Pellet output:

 http://www.w3.org/2002/07/owl#Thing
    onto.owl#Believe_Class0
    onto.owl#Believe_Class1
    onto.owl#Believe_Class2
    onto.owl#Class0 - (onto.owl#inst30_1, onto.owl#inst39_0, onto.owl#inst19_1, onto.owl#inst33_2, onto.owl#inst49_2, onto.owl#inst33_0, onto.owl#inst6_2, onto.owl#inst8_1, onto.owl#inst7_0, onto.owl#inst28_1, onto.owl#inst28_0, onto.owl#inst9_1, onto.owl#inst58_2, onto.owl#inst3_2, onto.owl#inst49_1, onto.owl#inst16_1, onto.owl#inst54_0, onto.owl#inst24_0, onto.owl#inst46_1, onto.owl#inst18_2, onto.owl#inst21_1, onto.owl#inst34_1, onto.owl#inst12_1, onto.owl#inst15_1, onto.owl#inst5_1, onto.owl#inst44_1, onto.owl#inst40_0, onto.owl#inst41_2, onto.owl#inst45_0, onto.owl#inst47_1, onto.owl#inst7_2, onto.owl#inst22_1, onto.owl#inst26_2, onto.owl#inst27_1, onto.owl#inst51_1, onto.owl#inst40_1, onto.owl#inst41_0, onto.owl#inst11_0, onto.owl#inst43_1, onto.owl#inst48_2, onto.owl#inst25_1, o

# 10. Перевод онтологии в CWA

In [13]:
with onto:
  for inst in onto.individuals():
    close_world(inst)

onto.save('reasoned_cwa.owl', format='ntriples')

# 11. Ризонер для классификации

In [14]:
with onto:
  sync_reasoner_pellet(debug=2)

onto.save('reasoned_cwa.owl', format='ntriples')

* Owlready2 * Running Pellet...
    java -Xmx2000M -cp /usr/local/lib/python3.8/dist-packages/owlready2/pellet/owlapi-distribution-3.4.3-bin.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-iri-0.9.5.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-tdb-0.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jcl-over-slf4j-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/jena-arq-2.10.0.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/aterm-java-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xml-apis-1.4.01.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/slf4j-api-1.6.4.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/commons-codec-1.6.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/pellet-2.3.1.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/httpcore-4.2.2.jar:/usr/local/lib/python3.8/dist-packages/owlready2/pellet/xercesImpl-2.10.0.jar:/usr/local/lib/p